In [1]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools, ConfluentTableDescriptor
from pyflink.table import TableEnvironment, DataTypes, Schema
from pyflink.table.expressions import *
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.window import Tumble, Slide, Session
from pyflink.table.udf import udf

In [2]:
from dotenv import load_dotenv
load_dotenv("./.env")

True

In [3]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

In [4]:
env.list_catalogs() # list Kafka environments

['default', 'examples', 'msds682']

In [5]:
env.use_catalog("msds682")

In [6]:
env.list_databases() # list Kafka clusters

['INFORMATION_SCHEMA', 'msds682']

In [7]:
env.use_database("msds682")

In [8]:
env.list_tables() # list Kafka topics

['system.usage',
 'system.usage$errors',
 'system.usage.flink',
 'system.usage.flink$errors',
 'system.usage.windowed',
 'system.usage.windowed$errors',
 'window_results',
 'window_results$errors']

In [9]:
table = env.from_path("`system.usage.flink`")

In [10]:
ConfluentTools.print_changelog_limit(table, 10)

+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+-------------------------+
| op |                            key |                             id |                      cpu_usage |                      cpu_stats |                   memory_usage |               timestamp |                $rowtime |
+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+-------------------------+-------------------------+
| +I |                          x'35' |                              5 |                            0.0 | [7846.0, 168477.0, 1.052838... |                           61.0 | 2025-11-20 18:41:21.980 | 2025-11-20 10:41:21.981 |
| +I |                          x'35' |                              5 |                           21.4 

In [11]:
result = table.where(col("id")=='2').select(col("id"),col("memory_usage"), col("$rowtime").alias("rowtime"))
ConfluentTools.print_materialized_limit(result, 10)

+----+--------------+-------------------------+
| id | memory_usage |                 rowtime |
+----+--------------+-------------------------+
|  2 |         61.0 | 2025-11-20 10:41:21.278 |
|  2 |         61.0 | 2025-11-20 10:41:21.783 |
|  2 |         61.0 | 2025-11-20 10:41:21.885 |
|  2 |         61.0 | 2025-11-20 10:41:21.929 |
|  2 |         61.0 | 2025-11-20 10:41:22.492 |
|  2 |         61.2 | 2025-11-20 10:42:19.735 |
|  2 |         61.1 | 2025-11-20 10:42:51.904 |
|  2 |         60.5 | 2025-11-20 10:51:21.978 |
|  2 |         60.6 | 2025-11-20 10:51:23.546 |
|  2 |         60.7 | 2025-11-20 10:51:41.121 |
+----+--------------+-------------------------+
10 rows in set


In [12]:
ConfluentTools.print_changelog_limit(result, 10)

+----+--------------------------------+--------------------------------+-------------------------+
| op |                             id |                   memory_usage |                 rowtime |
+----+--------------------------------+--------------------------------+-------------------------+
| +I |                              2 |                           61.0 | 2025-11-20 10:41:21.278 |
| +I |                              2 |                           61.0 | 2025-11-20 10:41:21.783 |
| +I |                              2 |                           61.0 | 2025-11-20 10:41:21.885 |
| +I |                              2 |                           61.0 | 2025-11-20 10:41:21.929 |
| +I |                              2 |                           61.0 | 2025-11-20 10:41:22.492 |
| +I |                              2 |                           61.2 | 2025-11-20 10:42:19.735 |
| +I |                              2 |                           61.1 | 2025-11-20 10:42:51.904 |
| +I |    